<a href="https://colab.research.google.com/github/matpg/NeuralNetAprendiendoTensor/blob/master/BreastCancerNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cargar el Dataset ##
En estas primeras lineas instalé tensorflow aunque venga por defecto e imprimo la version del tf.
En segundo lugar pido por al usuario los archivos .CSV para guardar el training data y el test data (esta dataset está previmiamente ordenada y normalizada). 
DATASET: Wisconsin breast cancer dataset
OUTPUT esperado: valores de 0 y 1(un 0 si es benigno y un 1 si es maligno).

In [48]:
import pandas as pd
import tensorflow as tf

from google.colab import files
file = files.upload()
X_train = pd.read_csv("xtrain.csv", header=None)
Y_train = pd.read_csv("ytrain.csv", header=None)
X_test = pd.read_csv("xtest.csv", header=None)
Y_test = pd.read_csv("ytest.csv", header=None)

Saving xtest.csv to xtest (1).csv
Saving xtrain.csv to xtrain (1).csv
Saving ytest.csv to ytest (1).csv
Saving ytrain.csv to ytrain (1).csv


En esta linea se puede apreciar algunos valores de entrenamiento.

In [49]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-1.150365,-0.390642,-1.128550,-0.958764,0.310984,-0.595995,-0.802596,-0.802490,0.294539,0.094251,-0.495052,1.487202,-0.514488,-0.491540,0.281498,-0.604512,-0.469007,-0.611700,0.057982,-0.357637,-1.043176,0.213533,-1.036045,-0.848808,0.342499,-0.730097,-0.812321,-0.757984,-0.016148,-0.385034
1,-0.937990,0.680514,-0.948201,-0.821525,-0.609636,-0.909867,-0.660669,-0.898716,0.754935,-0.425471,-0.333818,0.759412,-0.287518,-0.421277,-0.162080,-0.204867,-0.050296,-0.203091,-0.254690,-0.391395,-0.715654,1.066842,-0.689922,-0.668697,-0.095537,-0.537866,-0.375048,-0.606870,0.096690,-0.386158
2,0.574121,-1.033336,0.513941,0.408586,-0.106161,-0.363019,-0.417990,-0.088446,-0.271820,-0.575221,-0.576726,-1.057845,-0.538560,-0.387089,-1.072119,-0.720575,-0.423628,-0.492190,-0.674844,-0.801473,0.297615,-0.977818,0.262137,0.113888,-0.524724,-0.520866,-0.182989,-0.023719,-0.200502,-0.751443
3,-0.547220,-0.316022,-0.577622,-0.566615,0.586662,-0.649331,-0.805298,-0.500065,0.331078,0.540567,-0.128226,0.556222,-0.204001,-0.332347,-0.552851,-0.758881,-0.648914,0.601566,0.204548,-0.115963,-0.701325,-0.757927,-0.735737,-0.658966,-0.816748,-1.034921,-1.091633,-0.852545,-1.076186,-0.546883
4,-0.527398,0.791240,-0.561563,-0.523571,-1.051446,-1.017532,-0.905149,-0.935806,-0.969721,-0.426939,-0.628828,-0.130929,-0.613234,-0.466581,-0.671490,-0.744016,-0.710063,-1.204498,-0.542935,-0.503025,-0.427026,1.058637,-0.422423,-0.440955,-0.303494,-0.467251,-0.724565,-0.783118,0.311240,-0.082129


Aqui tenemos la representacion binaria de un tumor benigno o maligno

In [7]:
Y_train.head()

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


## Estructura de la Red ##
Usé Keras para definir el modelo de NeuralNet que voy a usar la cual va a ser entrenada con los training data para predecir futuros casos de cáncer de mama.
La red tiene una estructura de 4 Layers, la layer de entrada es de 16 neuronas de entrada, con la función de activación RElU, las 2 layers escondidas tiene 8 y 6 neuronas por capa respectivamente y la de salida 1 neurona con la función de activación sigmode que permite entregar un valor binario.

In [0]:
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential() # Initialising the ANN

classifier.add(Dense(units = 16, activation = 'relu', input_dim = 30))
classifier.add(Dense(units = 8, activation = 'relu'))
classifier.add(Dense(units = 6, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

Con este metodo, se puede ver una especie de informe describiendo la estructura de la NeuralNet configurada

In [51]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 16)                496       
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 7         
Total params: 693
Trainable params: 693
Non-trainable params: 0
_________________________________________________________________


### Definicion del Optimizador ###
: definida la estructura de la NeuralNet, se especifica el optimizador de esta, además del tipo de perdida y exactitud que se quiere.

In [0]:
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ["accuracy"])

### Entrenamiento de la Red ###
Ahora se entrena la red con los datos de entrada (x train) y sus respectivas labels (y train) para que la red aprenda a generalizar ejemplos, la red aprenderá los patrones entre los datos, iterará en 100 épocas como habitualmente se llama

In [53]:
classifier.fit(X_train, Y_train, batch_size = 1, epochs = 100 )




Epoch 1/100





455/455 [==============================] - 5s 11ms/step - loss: 0.2510 - acc: 0.9099
Epoch 2/100
455/455 [==============================] - 2s 5ms/step - loss: 0.1056 - acc: 0.9648
Epoch 3/100
455/455 [==============================] - 2s 5ms/step - loss: 0.0910 - acc: 0.9736
Epoch 4/100
455/455 [==============================] - 2s 5ms/step - loss: 0.0930 - acc: 0.9802
Epoch 5/100
455/455 [==============================] - 2s 5ms/step - loss: 0.0852 - acc: 0.9824
Epoch 6/100
455/455 [==============================] - 2s 5ms/step - loss: 0.0894 - acc: 0.9846
Epoch 7/100
455/455 [==============================] - 2s 5ms/step - loss: 0.0907 - acc: 0.9868
Epoch 8/100
455/455 [==============================] - 2s 5ms/step - loss: 0.1002 - acc: 0.9846
Epoch 9/100
455/455 [==============================] - 2s 5ms/step - loss: 0.0967 - acc: 0.9824
Epoch 10/100
455/455 [==============================] - 3s 6ms/step - loss: 0.0904 - acc: 0.9846
Epoch 11/100
455/455 [========

Con este metodo podemos conocer los parametros de perdida y exactitud que tiene nuestra NeuralNet, y así orientarnos sobre el rendimiento de nuestra configuracion previa.

In [56]:
classifier.evaluate(X_test, Y_test)
# primer valor: loss - perdida
# segundo valor: accuracy - exactitud

114/114 [==============================] - 0s 728us/step


[0.1943360811094368, 0.9649122775646678]

## Prediccion del modelo de NeuralNet ##
Ahora, para poder predecir nuevos valores, probamos un conjunto de datos no entrenados de prueba (x test) para predecir su label, o mejor dicho para que la NeuralNet sea capaz de dar en probabilidades y asignar que es lo más probable que sea comparado con el resultado real, si es que tiene un tumor benigno ( 1 dado por valores mayores o iguales a 0.5) o maligno (0 para valores menores a 0.5).

In [0]:
y_pred = classifier.predict(X_test)
y_pred = [ 1 if y>=0.5 else 0 for y in y_pred ]

Ahora probamos y vemos graficamente los resultados en comparacion con los reales, primero mostramos los resultados del 10 al 20 de la prediccion vs los resultados 10 al 20 de los reales.

In [58]:
for i in range(10,20):
  print(y_pred[i])

0
0
0
0
1
1
0
1
1
1


In [59]:
Y_test[10:20]

,0
10,0.0
11,0.0
12,0.0
13,0.0
14,0.0
15,1.0
16,0.0
17,1.0
18,1.0
19,1.0


### Resultados ###
Ahora se puede apreciar la cantidad total de casos correctos y erroneos en los cuales se acertó o no en la predicción, como se vé en los resultados, el error es minimo.

In [63]:
total, correct, wrong = 0, 0, 0
for i in range(len(y_pred)):
  total = total + 1
  if(Y_test.at[i,0] == y_pred[i]):
    correct = correct + 1
  else:
    wrong = wrong + 1

print("Total " + str(total))
print("Correct " + str(correct))
print("Wrong " + str(wrong))

Total 114
Correct 110
Wrong 4
